<a href="https://colab.research.google.com/github/taerri/meal-kit-analysis/blob/main/02_1_%EB%B9%85%EC%B9%B4%EC%9D%B8%EC%A6%88_%EB%B0%80%ED%82%A4%ED%8A%B8_%EC%A0%84%EC%B2%98%EB%A6%AC_Kkma(%EB%B3%B8_%EA%B3%BC%EC%A0%95%EC%9D%80_Colab%EC%9C%BC%EB%A1%9C_%EC%8B%A4%ED%96%89).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2. 데이터 전처리**

In [ ]:
pip install konlpy

In [ ]:
pip install customized_konlpy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from ckonlpy.tag import Twitter
from konlpy.tag import Okt
from konlpy.tag import Komoran
from collections import Counter
from wordcloud import WordCloud

In [ ]:
twitter=Twitter()

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
df=pd.read_csv("크롤링 결과.csv")
df.head()

,Unnamed: 0,Date,Title,Content
0,0,2017-12-27,"연말연시 ‘홈파티족’ 확산, 홈파티 ‘잇츠온’ 간편식 직접 체험해보니",요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1,1,2017-12-27,식물성·간편식·슈퍼곡물에 빠졌던 2017년,식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2,2,2017-12-27,"올해만 10여개 브랜드 출시, 가정간편식 '춘추전국시대'","야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나..."
3,3,2017-12-26,유통업계 '밀 솔루션' 경쟁 본격화,유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4,4,2017-12-19,"해먹남녀, GS리테일 ‘심플리쿡’ 판매 진행 “칼질 없이 일품요리 즐겨”",푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...


In [ ]:
#본문 리스트 만들기
word_list=[]
word_list=df['Content']
word_list=word_list.reset_index(drop=True)
word_list

0       요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1       식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2       야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나...
3       유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4       푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...
                              ...                        
5625    △편의점 GS25가 8월 한 달간 먹거리, 생필품 등을 전국 최저가 수준으로 선보이...
5626    [머니투데이 김건우 기자]\n코로나19(COVID-19) 재확산에 따른 사회적 거리...
5627    'AI, 빅데이터, 클라우드로 사회 기여하자'는 인식코로나19 사태로 어려움 겪는 ...
5628    [머니투데이 세종=안재용 기자]\n━\n◆국무총리실\n━\n\n\n2일(월)\n*김...
5629    [헤럴드경제=오연주 기자] 사회적 거리두기 장기화로 집밥 수요가 꾸준히 늘어나면서 ...
Name: Content, Length: 5630, dtype: object

In [ ]:
#기사중복 삭제
df_drop=df.drop_duplicates(['Date','Title','Content'], keep='first')

In [ ]:
#index reset
df=df_drop.reset_index(drop=True)

In [ ]:
#행,열 최대로 보기
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

In [ ]:
df.head()
len(df)

5507

In [ ]:
#기사 본문만 가져오기
content=df["Content"]
content.head()

0    요리하는 재미 ‘쏠쏠~~’ 맛도 ‘일품’ 분위기는 ‘덤’\n\n\n\n거리를 가득 ...
1    식품을 낭비없이 먹는 ‘제로 웨이스트’…SNS발달로 이국적 맛 ‘에스닉 푸드’ 등 ...
2    야구르트 아줌마가 배달하는 간편식,백화점의 프리미엄 간편식, 편의점에서 간단히 만나...
3    유통업계가 소비자 라이프스타일 변화에 발맞춰 간편한 한끼 식사가 가능한 제품과 서비...
4    푸드 콘텐츠 커머스 플랫폼 ‘해먹남녀’를 운영하는 바이탈힌트 코리아(대표 정지웅)는...
Name: Content, dtype: object

In [ ]:
#None값 제거
drop_row = content.dropna(axis=0)
df=pd.DataFrame([drop_row])
df= df.transpose()

In [ ]:
len(df)

5506

# **2-1 kkma 사용자 사전 추가**

In [ ]:
#사전에 단어 추가
twitter.add_dictionary('밀키트', 'Noun')
twitter.add_dictionary('코로나19', 'Noun')
twitter.add_dictionary('간편식', 'Noun')
twitter.add_dictionary('프레시지', 'Noun')
twitter.add_dictionary('식재료', 'Noun')
twitter.add_dictionary('GS리테일', 'Noun')
twitter.add_dictionary('한국야쿠르트', 'Noun')
twitter.add_dictionary('소상공인', 'Noun')
twitter.add_dictionary('심플리쿡', 'Noun')
twitter.add_dictionary('피코크', 'Noun')
twitter.add_dictionary('집밥', 'Noun')
twitter.add_dictionary('CJ제일제당', 'Noun')
twitter.add_dictionary('잇츠온', 'Noun')
twitter.add_dictionary('CJ', 'Noun')
twitter.add_dictionary('GS25', 'Noun')
twitter.add_dictionary('론칭', 'Noun')
twitter.add_dictionary('SSG닷컴', 'Noun')
twitter.add_dictionary('쿡킷', 'Noun')
twitter.add_dictionary('현대백화점', 'Noun')
twitter.add_dictionary('지속적', 'Noun')
twitter.add_dictionary('리테일', 'Noun')
twitter.add_dictionary('KT', 'Noun')
twitter.add_dictionary('1인', 'Noun')
twitter.add_dictionary('새벽배송', 'Noun')
twitter.add_dictionary('마켓컬리', 'Noun')
twitter.add_dictionary('SNS', 'Noun')
twitter.add_dictionary('신선식품', 'Noun')
twitter.add_dictionary('롯데백화점', 'Noun')
twitter.add_dictionary('대표적', 'Noun')
twitter.add_dictionary('편스토랑', 'Noun')
twitter.add_dictionary('온라인몰', 'Noun')
twitter.add_dictionary('기획전', 'Noun')
twitter.add_dictionary('언택트', 'Noun')
twitter.add_dictionary('마이셰프', 'Noun')
twitter.add_dictionary('집콕', 'Noun')
twitter.add_dictionary('본격적', 'Noun')
twitter.add_dictionary('현대그린푸드', 'Noun')
twitter.add_dictionary('시그니처', 'Noun')
twitter.add_dictionary('PB', 'Noun')
twitter.add_dictionary('11번가', 'Noun')
twitter.add_dictionary('RMR', 'Noun')
twitter.add_dictionary('식품업계', 'Noun')
twitter.add_dictionary('아이들', 'Noun')
twitter.add_dictionary('LG', 'Noun')

In [ ]:
#단어 교체(밀 키트 -> 밀키트)
content=df['Content'].map(lambda x: str(x).replace('밀 키트','밀키트'))
len(content)

5506

# **2-2 형태소 분석(명사 추출)**

In [ ]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
from tqdm import tqdm
kkma = Kkma()

In [ ]:
nouns = [] 
for x in tqdm(content):
    try:
        nouns.append(kkma.nouns(x))
    except:
        nouns.append(["None"])

100%|██████████| 5506/5506 [1:13:10<00:00,  1.25it/s]


In [ ]:
#명사 추출 결과값 열에 추가
konlpy_df= df.copy()
konlpy_df["ContentNoun"] = nouns

In [ ]:
#json 파일로 저장
konlpy_df.to_json("mealkit_전처리.json",orient="table")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import pickle
import re

In [ ]:
corpus = pd.read_json("mealkit_전처리.json",orient="table")

# **2-3 정규화**

In [ ]:
#단어 교체(11번 -> 11번가)
def change(text):
    text=','.join(text)
    text_list=text.split(',')
    result=[]
    for c in text_list:
        if c=='11번':
            c=c.replace('11번','11번가')
        if c=='인공지능':
            c=c.replace('인공지능','AI')
        result.append(c)
    result=','.join(result)
    return result

In [ ]:
corpus['ContentNoun']=corpus['ContentNoun'].map(lambda x: change(x))

# **2-4 한글자 제거**

In [ ]:
def remove(text):
    text_list=text.split(',')
    result=[]
    for n in text_list:
        if len(n)>1:
            result.append(n)
    result=','.join(result)
    return result

corpus['ContentNoun']=corpus['ContentNoun'].map(lambda x: remove(x))

# **2-5 불용어 처리**

## 불용어 제거
- 사이트 이용
    - [사이트1](https://bab2min.tistory.com/544)
    - [사이트2](https://www.ranks.nl/stopwords/korean)
- 직접 제거
    - ./삭제 단어.xlsx

In [ ]:
#직접 제거
stopword1 = pd.read_csv("./불용어 처리.txt",header=None,sep="\t")
stopword1 = list(stopword1[0])
len(stopword1)

139

In [ ]:
stopword2 = pd.read_csv("한국어불용어100.txt",header=None,sep="\t")
stopword2 = list(stopword2[0])
len(stopword2)

100

In [ ]:
stopword3 = pd.read_csv("한국어불용어100_2.txt",header=None)
stopword3 = list(stopword3[0])
len(stopword3)

675

In [ ]:
stopword = list(set(stopword1 + stopword2 + stopword3))
print("불용어 개수 :",len(stopword))

불용어 개수 : 803


In [ ]:
noun=corpus['ContentNoun']

In [ ]:
noun = [x for x in noun if x not in stopword]
print(noun[:1])

['요리,재미,일품,분위기,거리,인파,1시간,시간,기본,식당,카페,대기,대기시간,고가,크리스마스,특별,특별메뉴,메뉴,주문,가능,방침,등등,연말,모임,부담,생각,가성,가성비,추구,소비자,실속,의미,홈파티족,파티,확산,유통,유통업계,업계,최근,식품,매출,견인,가정,가정간편식,편식,홈파티,제품,그중,한국,한국야쿠르트,야쿠르트,잇츠온,간편,건강,재료,밀키트,키트,중심,다양,비밀,선물,포함,럭키,럭키박스,박스,출시,반응,일반적,레토르트,마감,매일,냉장,냉장상태,상태,배송,때문,풍미,장점,2차,조리,제대,기분,기자,이번,크리스마스이브,이브,친구,결정,체험,21,21일,오전,하이,하이프레시,다운로드,다운로드해,프라임,프라임스테이크,스테이크,쉬림프크림파스타,림프,크림,파스타,4만1900원,1900,감바스,바스,히요,1만6800원,6800,이틀,후인,23,23일,저녁,시간대,날짜,지정,단위,희망,선택,쿠르트,아줌마,배달,배달비,무료,빨간색,랜덤,3가지,가지,이상,커트,커트러리,크림치즈,치즈,브랜드,하루,하루야채,야채,마스크,마스크팩,콜드브,드브,커피,구미,젤리,풍족,우리,위장,평가,기준,3명,3개,무리,림프크림파스타,제외,고기,스파,거스,송이,송이버섯,버섯,토마토,양파,웨지,웨지감자,감자,크기,손질,해당,등급,척아이롤,아이,생고기,품질,마블링,군침,동봉,종이,스마트,스마트폰,정보,15,15분간,분간,4번,과정,완성,사진,의문,먼저,키친,키친타월,타월,핏물,제거,허브,허브솔트,솔트,오일,마리,이드,동안,10,10분,숙성,나머지,레시피상,피상,재고,램지에,지에,지금,큰소리,재촉,강불,앞뒤,1분,약불,3번,5분간,미디엄,의도,인덕션,화력,2분,정도,레시피,새우,브로콜리,베트남,베트남고추,고추,마늘,로즈,로즈메리,메리,모든,정량,음식물,쓰레기,접시,향초,와인,캐럴,레스토랑,만큼,마지막,버터,두께,식감도,감도,시들,조화,히요의,요의,사이즈,사이즈여,올리브,올리브오일,김동주,동주,마케팅,이사,1인,가구,증가,필요,만큼,식재료,인기,잇츠온,방식,신선도,유지,설명,7월,전국,판매,시작,올해

# **2-6 전처리 결과값**

In [ ]:
file=pd.DataFrame(noun, columns=['키워드'])
file.to_csv("단어(전처리 후).csv",encoding="utf-8-sig")

In [ ]:
file.head()

,키워드
0,"요리,재미,일품,분위기,거리,인파,1시간,시간,기본,식당,카페,대기,대기시간,고가,..."
1,"식품,낭비,제로,웨이스트,발달,이국적,에스,에스닉,푸드,트렌드,9가지,가지,건강,중..."
2,"야구르트,아줌마,배달,간편,간편식,백화점,프리미엄,편식,편의점,춘추,전국,시대,올해..."
3,"유통,유통업계,업계,소비자,라이프,라이프스타일,스타일,변화,식사,가능,제품,서비스,..."
4,"푸드,콘텐츠,머스,플랫폼,해먹,해먹남녀,남녀,운영,바이탈,바이탈힌트,힌트,코리아,대..."


### **※ 전처리 중 bigkinds에서 전처리 키워드를 제공하여, 이후부터는 해당 데이터(밀키트 뉴스데이터.csv)를 사용함**